In [1]:
import numpy as np 
import pandas as pd
import geopandas as gpd
import shapely
from shapely.wkt import loads
import shapely.vectorized as sv 
from scipy.spatial.distance import cdist 

In [2]:
# Load the nodes file of Alameda Island
nodes_df = pd.read_csv('../network_inputs/alameda_nodes.csv')

# Convert the nodes pandas dataframe into the geopandas dataframe
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry=[shapely.geometry.Point(xy) for xy in zip(nodes_df.lon, nodes_df.lat)], crs='epsg:4326')

# Find the node_id of the closest node to each parcel

In [3]:
# Load the parcel data
alameda_parcels = pd.read_csv('Parcels.csv')

# Extract parcel data of the Alameda Island and fill up the missing values
alameda_parcels = alameda_parcels[alameda_parcels['SitusZip'] == 94501] # The zip code of Alameda Island is 94501
alameda_parcels = alameda_parcels.fillna(method = 'ffill') # Fill the missing values by forward propagation
alameda_parcels['PARCEL'] = alameda_parcels['PARCEL'].astype(int)

# In this example, all parcels are treated in the same manner.
# However, the users can make more adjustments to the parcels data with the UseCode (https://propinfo.acgov.org/UseCodeList)

# Covert the pandas dataframe to the geopandas dataframe
household_parcels = alameda_parcels.copy()

household_parcels = gpd.GeoDataFrame(household_parcels, crs='epsg:4326',
                                     geometry=gpd.points_from_xy(household_parcels.xcoord, household_parcels.ycoord))
household_parcels['centroid'] = household_parcels['geometry']
household_parcels = household_parcels.set_geometry('centroid')

household_parcels['c_x'] = household_parcels['xcoord']
household_parcels['c_y'] = household_parcels['ycoord']

# Function to get the closest node to each parcel
def get_closest_node(parcel_x, parcel_y):
    return nodes_id[cdist([(parcel_x, parcel_y)], nodes_xy).argmin()]

# Find the closest node to each parcel
nodes_xy = np.array([nodes_gdf['geometry'].x.values, nodes_gdf['geometry'].y.values]).transpose()
nodes_id = nodes_gdf['node_id'].values

household_parcels['closest_node'] = household_parcels.apply(lambda x: get_closest_node(x['c_x'], x['c_y']), axis=1)

/var/folders/p5/yl_qhkks45xd154yl6qd7th40000gn/T/ipykernel_15801/860906012.py:2: DtypeWarning: Columns (3,5,7,8,19) have mixed types. Specify dtype option on import or set low_memory=False.
  alameda_parcels = pd.read_csv('Parcels.csv')


# Create the od pandas dataframe

In [4]:
# Set the closest node as the origin
local_residents_od_df = household_parcels[['PARCEL', 'closest_node']].copy().reset_index(drop=True)
local_residents_od_df['origin_nid'] = local_residents_od_df['closest_node']
local_residents_od_df = local_residents_od_df.drop('closest_node', 1)
local_residents_od_df['departure_time'] = 0 # It is assumed that all local resident evacuate at the same time
local_residents_od_df['type'] = 'local'

/var/folders/p5/yl_qhkks45xd154yl6qd7th40000gn/T/ipykernel_15801/2994937350.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  local_residents_od_df = local_residents_od_df.drop('closest_node', 1)


In [5]:
# Input the evacuation node_id here (it is assumed that all residents evacuate to 1 evacuation point)
# Set the evacuation node as the destination
local_residents_od_df['destin_nid'] = '734'

In [6]:
# Save the dataframe to csv file
local_residents_od_df.to_csv('local_residents_od.csv')